In [1]:
## load all the problems from the verilog-eval dataset

from verilog_eval.data import write_jsonl, read_problems, stream_jsonl
from tqdm import tqdm
import itertools
from verilog_eval.data import VERILOG_EVAL_HUMAN, HUMAN_DESCRIPTIONS

problems = read_problems(VERILOG_EVAL_HUMAN)
descriptions = read_problems(HUMAN_DESCRIPTIONS)
# machine_problems = read_problems("../ext/verilog-eval/data/VerilogEval_Machine.jsonl")
# machine_descriptions = read_problems("../ext/verilog-eval/descriptions/VerilogDescription_Machine.jsonl")

## truncate the problems
# problems = itertools.islice(problems.items(), 0, 10)
# problems = dict(problems)
# descriptions = {k: v for k, v in descriptions.items() if k in problems}
# write_jsonl("sample_problems.jsonl", list(problems.values()))

problems = list(problems.values())

In [ ]:
problems

In [ ]:
from vgen.interpreter import evaluate_code

In [ ]:
problems

In [ ]:
task_id = "gatesv"
comp = problems[task_id]["canonical_solution"]

In [ ]:
res = evaluate_code(task_id, comp, problems)

In [ ]:
res

In [3]:
import outlines

In [10]:
STOP_SEQUENCES = ["```"]

def split_problem_tests(problem):
    pre_base_str, tests = problem["test"].split("def check(candidate):\n")
    base_str = "def check(candidate):\n"
    split_tests = []
    # NOTE: assumes human-eval-specific logic for multiline asserts & for-loops
    # won't work properly if multiline assert nested within for-loop
    multiline_assert, parts = False, []
    for_loop, fl_parts = False, []
    for i in tests.split("\n"):
        if multiline_assert:
            parts.append(i)
            if i.lstrip().startswith("]"):
                test = "\n".join(parts)
                split_tests.append(pre_base_str + base_str + test)
                multiline_assert = False
        elif i.lstrip().startswith("assert") and i.lstrip()[-1] == "[":
            multiline_assert = True
            parts = [i]
        elif for_loop:
            fl_parts.append(i)
            if i.lstrip().startswith("assert"):
                test = "\n".join(fl_parts)
                split_tests.append(pre_base_str + base_str + test)
                for_loop, fl_parts = False, []
        elif (
            (i.lstrip() == "")
            or (i.lstrip().startswith("#"))
            or (i.lstrip().startswith("print"))
        ):
            continue
        elif not (i.lstrip().startswith("assert")):
            fl_parts.append(i)
            if i.lstrip().startswith("for"):
                for_loop = True
        # special logic for HumanEval/151
        elif problem["task_id"] == "HumanEval/151" and (
            i.lstrip().startswith("assert candidate(lst)")
        ):
            fl_parts.append(i)
            test = "\n".join(fl_parts)
            split_tests.append(pre_base_str + base_str + test)
            fl_parts = []
        else:
            split_tests.append(pre_base_str + base_str + i)
    return split_tests


# task_id_problem_map = {problem["task_id"]: problem for problem in problems}
# task_id_split_tests_map = {
    # problem["task_id"]: split_problem_tests(problem) for problem in problems
# }


def stats_execute(task_id, completion, timeout=10):
    problem = task_id_problem_map[task_id]
    split_tests = task_id_split_tests_map[task_id]
    thread_problems = [{**problem, "test": test} for test in split_tests]
    results = []
    with ThreadPoolExecutor() as executor:
        for result in executor.map(
            lambda tp: check_correctness(tp, completion, timeout), thread_problems
        ):
            results.append(result["passed"])

    return {
        "task_id": task_id,
        "pass_rate": sum(results) / len(results),
    }


@outlines.prompt
def few_shot_prompt(instructions, examples, description, question):
    """{{ instructions }}

    {% for example in examples %}
    Description:
    ```
    {{ example.description }}
    ```
    Question:
    ```
    {{ example.prompt }}
    ```
    Answer:
    ```
    {{ example.canonical_solution }}
    ```
    {% endfor %}

    Description:
    ```
    {{ description }}
    ```
    Question:
    ```
    {{ question }}
    ```
    Answer:
    ```
    """


instructions = "Please answer the following question following the examples. Generate valid verilog code always."
examples = problems[:2]


def get_prompts_with_ids():
    prompts_with_ids = [
        (few_shot_prompt(instructions, examples, descriptions[problem["task_id"]]['detail_description'], problem["prompt"]), problem["task_id"])
        for problem in problems[2:]
    ]
    return prompts_with_ids


In [11]:
# problems
# descriptions

In [12]:
get_prompts_with_ids()

UndefinedError: 'dict object' has no attribute 'description'